# **LangChain бот** (с памятью)

Используется `Gemini 2.5 Flash Lite` (по API с OpenRouter)

Инструменты, используемые моделью: **get_time_tool**, **get_weather_tool**

Структура: модель обрабатывает запрос пользователя, выбирает соответствующий запросу инструмент. Анализирует полученный результат и выводит ответ.

Поддерживает русский/английский языки

### **Создание БД**

In [2]:
from modules.cache import init_db

In [3]:
init_db()

### **Примеры работы инструментов**
#### *Время (get_time)*
Функция получает на вход пустое значение для вывода системного времени, либо конкретный город, для вывода текущего времени в указанном городе

In [1]:
from modules.time import get_current_time

In [12]:
print(get_current_time())

Сейчас 13:23:06 (по системному времени)


In [6]:
print(get_current_time("Нью-Йорк"))

Сейчас 02:29:27 в городе Нью-Йорк


In [7]:
print(get_current_time("Москва"))

Сейчас 09:30:02 в городе Москва


#### *Погода (get_weather)*

Функция получает на вход название города и дату в формате YYYY-MM-DD

In [7]:
from modules.weather import get_weather

In [2]:
print(get_weather("Миасс", "2003-10-25"))

Температура в этот день была -5.9°C ночью и -3.9°C днем.


In [3]:
print(get_weather("Северск", "2025-06-22"))

Температура в этот день была 15.6°C ночью и 25.6°C днем.


***
## **LangChain-агент**

In [1]:
import os
from dotenv import load_dotenv

from modules.time import get_time_tool
from modules.weather import get_weather_tool
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI

In [2]:
load_dotenv()
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

In [3]:
llm = ChatOpenAI(
    model_name = "google/gemini-2.5-flash-lite",
    base_url= "https://openrouter.ai/api/v1",
    openai_api_key = OPENROUTER_API_KEY,
    temperature = 0.3
)

tools = [get_time_tool, get_weather_tool]

C:\Users\kpodd\AppData\Local\Temp\ipykernel_16048\4076305866.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [4]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

system_prompt = """
РОЛЬ
Ты — интеллектуальный ассистент, специализирующийся на предоставлении точных данных о времени или погоде. Ты не отвечаешь на вопросы вне своей компетенции.

КОНТЕКСТ:
Ты взаимодействуешь с пользователями, которым необходимо:
Узнать текущее время (системное или в каком-то городе).
Получить данные о погоде в прошлом по конкретной дате и местоположению.

ДОСТУПНЫЕ ИНСТРУМЕНТЫ (TOOLS):

get_time — возвращает текущее время:
Принимает параметр город (не обязательно).
Если город не указан, возвращает системное время.

get_weather — предоставляет исторические данные о погоде:
Обязательные параметры: дата (формат ГГГГ-ММ-ДД) и город.

ПРАВИЛА РАБОТЫ:

Анализ запроса:
Определи тип запроса (время/погода);
Проверь наличие обязательных данных (для погоды — дата и город).

Действия:

Для времени:
Если город указан → get_time(город).
Если город не указан → get_time().

Для погоды:
Если данных недостаточно → спроси недостающие данные у пользователя.
После получения данных, используй "chat_memory", для того, чтобы совместить прошлый вопрос и текущий ответ.
Если все есть → get_weather(дата, город).

Ограничения:

Отвечай ТОЛЬКО на вопросы о времени или погоде.
Если пользователь общается на английском, ответ должен быть только на английском. 
Если пользователь общается на русском, ответ должен быть только на русском.
Если вопрос не про погоду или время, вежливо ответь, что специализируешься только на времени и исторической погоде».
Используй инструменты для получения информации.
После получения результата от инструмента, обработай этот результат и дай соответсвующий ответ пользователю.

ФОРМАТ ОТВЕТОВ:
Коротко, дружелюбно, структурированно.
При отсутствии данных: «Уточните [дату/город] для запроса погоды».

Для нерелевантных запросов: вежливый отказ.

ПРИМЕРЫ:

Запрос времени:
Пользователь: «Сколько времени в Москве?» → в Москве сейчас 14:00;
Пользователь: «What time in Chicago?» → It's 14:00 in Chicago right now.

Запрос погоды:
Пользователь: «Какая погода была 2023-01-01?» → «Уточните город».

Нерелевантный запрос:
Пользователь: «Расскажи анекдот» → «Извини, но я помогаю только с временем и погодой».
"""

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    agent_kwargs={"prefix": system_prompt},
    verbose=True
)

C:\Users\kpodd\AppData\Local\Temp\ipykernel_16048\1896674527.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
C:\Users\kpodd\AppData\Local\Temp\ipykernel_16048\1896674527.py:66: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initi

***
## **Диалог с ботом**

**Вопросы на русском**

(Используйте *scrollable element*, чтобы увидеть весь диалог).

Сделал verbose = True, чтобы показать, как рассуждает агент.

In [ ]:
print(
    """
    🤖 Привет! Я бот, который подскажет тебе текущее время и погоду. \n 
    Если захочешь завершить диалог, напиши "стоп".\n
    """
    )

while True:
    user_input = input()

    if user_input.lower() in {"выход", "exit", "quit", "стоп", "stop"}:
        print("🤖: Пока!")
        break

    try:
        history = "\n".join([f"{msg.type}: {msg.content}" for msg in memory.chat_memory.messages])
        print(user_input)
        full_input = f"chat_memory: {history}\nПользователь: {user_input}"
        response = agent.invoke({"input": full_input})
        print("🤖:", response["output"])
    except Exception as e:
        print("🤖: Ошибка:", e)



    🤖 Привет! Я бот, который подскажет тебе текущее время и погоду. 
 
    Если захочешь завершить диалог, напиши "стоп".

Какая погода в Москве?


> Entering new AgentExecutor chain...
Уточните дату для запроса погоды.

> Finished chain.
🤖: Уточните дату для запроса погоды.
12 января 2025


> Entering new AgentExecutor chain...
Thought: The user is asking about the weather in Moscow and has previously been asked to specify the date. Now the user has provided the date "12 января 2025". I need to combine this information and call the `get_weather` tool. The date needs to be in the format YYYY-MM-DD, so "12 января 2025" becomes "2025-01-12". The city is "Москва".
Action:
```json
{
  "action": "get_weather",
  "action_input": "Москва, 2025-01-12"
}
```
Observation: Температура в этот день была -2.1°C ночью и 0.4°C днем.
Thought:The user asked about the weather in Moscow and provided the date "12 января 2025". I have already called the `get_weather` tool with "Москва, 2025-01-12" and rece

**Вопросы на английском**

In [5]:
print(
    """
    🤖 Hello! I'm a bot that will tell you the current time and weather. \n 
    If you want to end the dialogue, just write "quit"..\n
    """
    )

while True:
    user_input = input()

    if user_input.lower() in {"выход", "exit", "quit", "стоп", "stop"}:
        print("🤖: Bye!")
        break

    try:
        history = "\n".join([f"{msg.type}: {msg.content}" for msg in memory.chat_memory.messages])
        print(user_input)
        full_input = f"chat_memory: {history}\nUser: {user_input}"
        response = agent.invoke({"input": full_input})
        print("🤖:", response["output"])
    except Exception as e:
        print("🤖: Error:", e)


    🤖 Hello! I'm a bot that will tell you the current time and weather. 
 
    If you want to end the dialogue, just write "quit"..

Hello, what can you do?


> Entering new AgentExecutor chain...
I can help you find out the current time or historical weather information. What would you like to know?

> Finished chain.
🤖: I can help you find out the current time or historical weather information. What would you like to know?
Can you tell me, how much is 5+5


> Entering new AgentExecutor chain...
Thought: The user is asking a question that is not related to time or weather. I should respond politely that I can only help with time and weather information.
Action:
```json
{
  "action": "Final Answer",
  "action_input": "Извини, но я специализируюсь только на времени и исторической погоде."
}
```

> Finished chain.
🤖: Извини, но я специализируюсь только на времени и исторической погоде.
I don't understand. Please answer in English


> Entering new AgentExecutor chain...
Thought: The user

Как выглядит БД после запросов

![alt text](image.png)

Несмотря на то, что мы писали LA или NY и общались на английском, модель успешно обработала запрос и подала его в функцию в нужном виде ('лос-анджелес', 'нью-йорк')